In [2]:
# 0314 v2 - 가까운 두 개 마커로 위치 파악
# 마커 개수 늘림 시야에 3개까지 잘 잡히는데 가까운 2개를 고르려다 보니까 어떤 마커 2개를 고르는지에 따라서 좌표가 다르게 나타남
#v3: 여러개 중에 2개씩 이용해서 위치 뽑고 평균 내기
#v4: 칼만 필터  - v3보다 개선됨 (튀는 값이 줄어들었지만 여전히 많긴 함)

In [1]:
import cv2
import cv2.aruco as aruco
import numpy as np

# 1) 카메라 내부 파라미터 로드
cameraMatrix = np.load('C:/cal_cam/calibration_matrix.npy')
distCoeffs = np.load('C:/cal_cam/distortion_coefficients.npy')

# 2) 마커 실제 한 변 길이 (단위 m)
marker_length = 0.16

# 3) ArUco 사전, 파라미터 생성
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
detector_params = aruco.DetectorParameters()

# 4) 월드 좌표에서 마커의 위치(0~16번)
marker_world_pos = {
    0: np.array([0.0, 0.0, 0.0]), 1: np.array([1.5, 0.0, 0.0]),
    2: np.array([3.0, 0.0, 0.0]), 3: np.array([4.5, 0.0, 0.0]),
    4: np.array([0.0, 1.5, 0.0]), 5: np.array([1.5, 1.5, 0.0]),
    6: np.array([3.0, 1.5, 0.0]), 7: np.array([4.5, 1.5, 0.0]),
    8: np.array([0.0, 3.0, 0.0]), 9: np.array([1.5, 3.0, 0.0]),
    10: np.array([3.0, 3.0, 0.0]), 11: np.array([4.5, 3.0, 0.0]),
    12: np.array([0.7, 0.7, 0.0]), 13: np.array([2.3, 0.7, 0.0]),
    14: np.array([3.7, 0.7, 0.0]), 15: np.array([0.7, 2.3, 0.0]),
    16: np.array([2.3, 2.3, 0.0]),
}

# 칼만 필터 초기화 함수
def init_kalman_filter():
    kf = cv2.KalmanFilter(6, 3)  # 3D 위치 + 속도
    kf.measurementMatrix = np.eye(3, 6, dtype=np.float32)
    kf.transitionMatrix = np.array([
        [1, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ], dtype=np.float32)
    kf.processNoiseCov = np.eye(6, dtype=np.float32) * 1e-4
    kf.measurementNoiseCov = np.eye(3, dtype=np.float32) * 1e-2
    return kf

# 마커 개별 칼만 필터 저장
kalman_filters = {}

# 카메라 위치 계산
def compute_camera_pos(marker_id, rvec, tvec, marker_world):
    R, _ = cv2.Rodrigues(rvec)
    t = tvec.reshape(3, 1)
    cam_in_marker = -R.T @ t
    return marker_world.reshape(3, 1) + cam_in_marker

# 5) 카메라 열기
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_EXPOSURE, -6)

if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=detector_params)

    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, marker_length, cameraMatrix, distCoeffs)

        id_to_pose = {id_[0]: (rvecs[i], tvecs[i]) for i, id_ in enumerate(ids)}
        found_ids = ids.flatten()

        cam_positions = []
        weights = []

        for m_id in found_ids:
            if m_id in marker_world_pos:
                rvec, tvec = id_to_pose[m_id]
                world_pos = marker_world_pos[m_id]
                cam_pos = compute_camera_pos(m_id, rvec, tvec, world_pos).flatten()

                # 칼만 필터 초기화/업데이트
                if m_id not in kalman_filters:
                    kalman_filters[m_id] = init_kalman_filter()

                kf = kalman_filters[m_id]
                kf.correct(cam_pos.astype(np.float32))  # 관측으로 보정
                estimate = kf.predict()[:3]  # 예측 값 사용

                cam_positions.append(estimate)
                distance = np.linalg.norm(tvec)
                weights.append(1.0 / (distance + 1e-6))  # 가까운 마커 가중치 높게

                # 마커 ID 표시
                c = corners[found_ids.tolist().index(m_id)][0][0]
                cv2.putText(frame, f"ID {m_id}", (int(c[0]), int(c[1] - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        if cam_positions:
            weights = np.array(weights)
            weights /= np.sum(weights)  # 정규화
            cam_positions = np.array(cam_positions)

            # 가중 평균으로 최종 위치
            camera_world_pos = np.average(cam_positions, axis=0, weights=weights)


            cx, cy, cz = camera_world_pos.ravel()
            print(f"[INFO] Camera world position (filtered): ({cx:.2f}, {cy:.2f}, {cz:.2f})")



            #cx, cy, cz = camera_world_pos
            #print(f"[INFO] Camera world position (filtered): ({cx:.2f}, {cy:.2f}, {cz:.2f})")

    # 화면 표시
    cv2.imshow("Kalman Filter ArUco Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC 종료
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Camera world position (filtered): (0.00, 0.00, 0.00)
[INFO] Camera world position (filtered): (0.05, 0.05, 0.02)
[INFO] Camera world position (filtered): (0.22, 0.25, 0.11)
[INFO] Camera world position (filtered): (0.71, 0.80, 0.34)
[INFO] Camera world position (filtered): (1.61, 1.84, 0.79)
[INFO] Camera world position (filtered): (2.82, 3.24, 1.36)
[INFO] Camera world position (filtered): (4.02, 4.61, 1.98)
[INFO] Camera world position (filtered): (4.96, 5.65, 2.56)
[INFO] Camera world position (filtered): (5.54, 6.31, 2.93)
[INFO] Camera world position (filtered): (5.84, 6.67, 3.12)
[INFO] Camera world position (filtered): (6.02, 6.76, 3.29)
[INFO] Camera world position (filtered): (5.96, 6.73, 3.28)
[INFO] Camera world position (filtered): (5.85, 6.60, 3.21)
[INFO] Camera world position (filtered): (5.66, 6.40, 3.14)
[INFO] Camera world position (filtered): (5.45, 6.17, 3.07)
[INFO] Camera world position (filtered): (5.25, 5.97, 2.94)
[INFO] Camera world position (filtered):